In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import math
import datetime
import re
import time
import os
import json
from PIL import Image 
import PIL 
pd.set_option('display.max_columns', None)
import random
import codecs
import subprocess
from selenium import webdriver
from bs4 import BeautifulSoup
import chromedriver_binary
from selenium.webdriver.chrome.options import Options

# Headers

In [2]:
headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}

In [497]:
full_occaz_brand_list = ["AC+EMOTION",
                         "ACCESS+MOTOR",
                         "APRILIA",
                         "ARCTIC+CAT",
                         "BENELLI",
                         "BETA",
                         "BIMOTA",
                         "BMW",
                         "BUELL",
                         "CAN-AM",
                         "CF+MOTO",
                         "DAELIM",
                         "DERBI",
                         "DUCATI",
                         "FANTIC",
                         "FB+MONDIAL",
                         "GAS+GAS",
                         "GENERIC",
                         "GILERA",
                         "HARLEY+DAVIDSON",
                         "HARLEY-DAVIDSON",
                         "HER+CHEE",
                         "HM",
                         "HONDA",
                         "HONGYI",
                         "HUSQVARNA",
                         "HYOSUNG",
                         "HYTRACK",
                         "IMF+scooter",
                         "INDIAN",
                         "IRBIT",
                         "JM+MOTORS",
                         "JORDON",
                         "JOTAGAS",
                         "KAWASAKI",
                         "KEEWAY",
                         "KSR+MOTO",
                         "KTM",
                         "KYMCO",
                         "LAMBRETTA",
                         "LAZIO",
                         "LIGIER",
                         "LINHAI",
                         "LONGJIA",
                         "Magnum",
                         "MAGPOWER",
                         "MALAGUTI",
                         "MARTIN",
                         "MASAI",
                         "MASH",
                         "MBK",
                         "MOTO+MORINI",
                         "MOTO-GUZZI",
                         "MOTOCONFORT",
                         "MOTRAC",
                         "MV+AGUSTA",
                         "NECO",
                         "NIU",
                         "NORTON",
                         "ORCAL",
                         "PEUGEOT",
                         "PIAGGIO",
                         "POLARIS",
                         "QUADDY",
                         "QUADRO",
                         "RIEJU",
                         "RIVAL+MOTORS",
                         "RIYA",
                         "ROYAL+ENFIELD",
                         "SHERCO",
                         "SUPER+SOCO",
                         "SUZUKI",
                         "SVM+%28SWM%29",
                         "SWM",
                         "SYM",
                         "TGB",
                         "TNT+MOTOR",
                         "TRIUMPH",
                         "VASTRO",
                         "VESPA",
                         "Victory+Motorcycle",
                         "VOGE",
                         "VOXAN",
                         "WANGYE",
                         "XINGYUE",
                         "YAMAHA",
                         "ZERO+MOTORCYCLES"]   

# Save pages

In [3]:
def save_page_list(req, page=1):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_list_name = "fulloccaz" + "-" + datetime_1 + "-" + str(page)
    with open("pages/" + page_list_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()
    return "pages/" + page_list_name + ".html"

In [5]:
#def get_source_selenium(url_):
#    options = Options()
#    options.add_argument("--headless")
#    driver = webdriver.Chrome(options=options, executable_path="/Users/victor/Downloads/chromedriver")
#    driver.get(url_)
#    page_source = driver.page_source
#    driver.quit()
#    return page_source

In [ ]:
scrap = 1
page_=1
page_total = 1
#
for motobrand in full_occaz_brand_list: #https://www.fulloccaz.com/?&Marque=HONDA&page=1
    scrap = 1
    page_=1
    while scrap==1:
        html_file = requests.get(f"https://www.fulloccaz.com/?&Marque={motobrand}&page={page_}", headers=headers)
        last_saved_file = save_page_list(html_file, page=page_total)
        
        with open(last_saved_file, 'r') as f:
            readable_html = f.read()
            prices = get_prices_from_pages(readable_html)
        if len(prices) == 0:
            scrap=0
        
        page_ += 1
        page_total += 1    
        time.sleep(random.randint(1, 3))
        print("page saved: " + str(page_total))

# Extract data from pages

## Test file

In [4]:
file_test_name = [file for file in os.listdir("pages") if file.endswith("html")][-2]
file_test_name

'fulloccaz-2021-06-05_18h16-27.html'

In [5]:
file_check_x15_good = f"pages/{file_test_name}"
file_check_x15_good

'pages/fulloccaz-2021-06-05_18h16-27.html'

## Get prices from pages

In [6]:
def get_prices_from_pages(r):
    
    # set empty price list
    price_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[id*="IDVignettes"]')
    if len(step1) > 0:
        step2 = step1[0].select('div[class*="CLPrix"]')
        if len(step2) > 0:
            for k in step2:
                try:
                    price_list.append(float("".join(re.findall("[0-9]", k.text.replace("\n", "").replace("\t", "")))))
                except:
                    price_list.append(np.nan)    
    return price_list

In [7]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    prices = get_prices_from_pages(readable_html)
prices

[6490.0,
 9590.0,
 5290.0,
 10790.0,
 13590.0,
 1290.0,
 7990.0,
 4500.0,
 3490.0,
 8999.0,
 1350.0,
 5590.0]

## Get url from pages

In [8]:
def get_urls_from_pages(r):
    
    # set empty url list
    url_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[id*="IDVignettes"]')
    if len(step1) > 0:
        step2 = step1[0].select('a')
        if len(step2) > 0:
            for k in step2:
                if "href" in k.attrs:
                    try:
                        url_list.append("https://www.fulloccaz.com" + k.attrs["href"])
                    except:
                        url_list.append(np.nan)    
    return url_list

In [9]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    urls = get_urls_from_pages(readable_html)
urls

['https://www.fulloccaz.com/fiches/index/2028161-occasion-yamaha-fjr-1300a?',
 'https://www.fulloccaz.com/fiches/index/2015502-occasion-kawasaki-versys-650-se-gt?',
 'https://www.fulloccaz.com/fiches/index/2033933-occasion-kymco-400-xciting-s?',
 'https://www.fulloccaz.com/fiches/index/2049541-occasion-triumph-street-triple-765-rs?',
 'https://www.fulloccaz.com/fiches/index/2033378-occasion-yamaha-niken-850?',
 'https://www.fulloccaz.com/fiches/index/2030313-occasion-kymco-zing-125-zing125?',
 'https://www.fulloccaz.com/fiches/index/2049506-occasion-kymco-ak-550-abs-e4-7990?',
 'https://www.fulloccaz.com/fiches/index/2048061-occasion-300-mp3?',
 'https://www.fulloccaz.com/fiches/index/2040258-occasion-honda-cbf-1000-f?',
 'https://www.fulloccaz.com/fiches/index/2049473-occasion-yamaha-tracer-900-gt?',
 'https://www.fulloccaz.com/fiches/index/2035911-occasion-yamaha-majesty-125?',
 'https://www.fulloccaz.com/fiches/index/2016668-occasion-kymco-xciting-400i-s?']

## Get references from pages

In [10]:
def get_references_from_url_list(url_list_):
    
    # set empty ref list
    ref_list = []
    
    for k in url_list_:
        try:
            ref_list.append(k.split("/")[5].split("-")[0])
        except:
            ref_list.append(np.nan)
    #                
    return ref_list

In [11]:
get_references_from_url_list(urls)

['2028161',
 '2015502',
 '2033933',
 '2049541',
 '2033378',
 '2030313',
 '2049506',
 '2048061',
 '2040258',
 '2049473',
 '2035911',
 '2016668']

# Download single announce pages

In [12]:
data_fulloccaz = "../../../tresboncoin/data/fulloccaz.csv"
#data = pd.read_csv(data_motoplanete)

In [13]:
def add_the_announce(df, uniq_id, price):
    if int(uniq_id) in list(df["unique id"]):
        index_ = df.index[df['unique id'] == int(uniq_id)].tolist()[0]
        
        # if an announce with same uniq id and same price is found, return true and skip
        return df.iloc[index_]["price"]!=price
    
    # else, return false and add the announce
    return True

In [14]:
def save_page_uniq(req, uniq_id):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_name = "fulloccaz" + "-" + uniq_id + "-" + datetime_1
    with open("annonces/" + page_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [15]:
def file_exists(uniq_id):
    for k in os.listdir("annonces"):
        if "-" + str(uniq_id) + "-" in k:
            return True
    return False

In [304]:
#qq = "https://www.fulloccaz.com/fiches/index/2016930-occasion-piaggio-mp3-300-hpe-abs-asr?"
#req_uniq = requests.get(qq, headers = headers)
#save_page_uniq(req_uniq, "2016930")

In [ ]:
announce_count = 1
page_count = 0
for html_file in [file for file in os.listdir("pages") if file.endswith("html")]:
    page_count +=1
    print("page n°: " + str(page_count))
    with open(f"pages/{html_file}", 'r') as f:
        readable_html = f.read()
        urls_list = get_urls_from_pages(readable_html)
        unique_id = get_references_from_url_list(urls_list)
        prices_list = get_prices_from_pages(readable_html)
    if urls_list != None:
        for url_single, uniq_id, price_ in zip(urls_list, unique_id, prices_list):
            
            #if add_the_announce(data_price, uniq_id, price_):
            #html_file = get_source_selenium(url_single)
            if file_exists(uniq_id) == False:
                req_uniq = requests.get(url_single, headers = headers)
                save_page_uniq(req_uniq, uniq_id)
                print(str(announce_count) + ". saved page id: " + uniq_id)
                time.sleep(random.randint(1, 5))
                announce_count += 1
            else:
                print(announce_count, url_single + " --> skipped")
                announce_count += 1

# Process single announces
Saving scaled images x3 and adding announce to DataFrame

In [438]:
announce_template = pd.DataFrame({"url": [np.nan],
                                  "unique id": [np.nan],
                                  "date_scrapped": [np.nan],
                                  "announce_publication_date": [np.nan],
                                  "vehicle brand": [np.nan],
                                  "vehicle type": [np.nan],
                                  "color": [np.nan],
                                  "price": [np.nan],
                                  "city": [np.nan],
                                  "postal code": [np.nan],
                                  "seller": [np.nan],
                                  "seller_name": [np.nan],
                                  "vehicle release date": [np.nan],
                                  "mileage": [np.nan],
                                  "motorisation": [np.nan],
                                  "fiscal power": [np.nan],
                                  "guarantee": [np.nan],
                                  "hand": [np.nan],
                                  "chassis": [np.nan],
                                  "engine capacity [CC]": [np.nan],
                                  "comments": [np.nan]})
announce_template

,url,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,color,price,city,postal code,seller,seller_name,vehicle release date,mileage,motorisation,fiscal power,guarantee,hand,chassis,engine capacity [CC],comments
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## test announce

In [713]:
file_name = "fulloccaz-2041222-2021-06-07_22h37.html"#os.listdir("annonces")[-1]
#
file_check_single = f"annonces/{file_name}"
file_check_single

'annonces/fulloccaz-2041222-2021-06-07_22h37.html'

## Functions

### Get url

In [714]:
def get_url(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('a[class*="CLFacebook"]')
    if len(step1)>0:
        if "href" in step1[0].attrs:
            return step1[0].attrs["href"].split("=")[-1]
    return np.nan

In [715]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
g_url

'http://www.fulloccaz.com/fiches/index/2041222-occasion-bmw-c-evo'

### Get unique id

In [716]:
def get_uniq_id(url_):
    try:
        return url_.split("/")[-1].split("-")[0]
    except:
        return np.nan

In [717]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
get_uniq_id(g_url)

'2041222'

### get publication date

In [718]:
def get_publication_date(r):

    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Mise à jour") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [719]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_pub = get_publication_date(readable_html)
g_pub

'30/05/2021'

### Get vehicule brand

In [720]:
def get_brand_and_model(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('h1')
    if len(step1)>0:
        return step1[0].text
    else:
        return None

In [721]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_brand = get_brand_and_model(readable_html)
g_brand

'BMW C EVO'

### Get vehicule type

In [722]:
def get_type(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Catégorie") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [723]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_type(readable_html)
g_type

'Scooter'

### get color

In [724]:
def get_color(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Couleur") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [725]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_color(readable_html)
g_type

'Argent'

### get price

In [726]:
def get_price(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')

    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Prix") >= 0:
                    try:
                        return float("".join(re.findall("\d+", k.select('td')[1].text)))
                    except:
                        return np.nan
    else:
        return np.nan

In [727]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_price(readable_html)
g_type

12200.0

### get city

In [835]:
def get_city(r):
    
    # get soup
    soup = BeautifulSoup(r, features="lxml")
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceConcession"]')
    if len(step1)>0:
        try:
            return step1[0].text.split(re.findall("[0-9]{5}", step1[0].text)[0])[-1].strip()
        except:
            return np.nan
    return None

In [836]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_city = get_city(readable_html)
g_city

'LANESTER'

### get postal code

In [829]:
def get_postal(r):
    
    # get soup
    soup = BeautifulSoup(r)
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceConcession"]')
    if len(step1)>0:
        try:
            return re.findall("[0-9]{5}", step1[0].text)[0]
        except:
            return np.nan
    return np.nan

In [830]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_post = get_postal(readable_html)
g_post

'56600'

### get seller

In [777]:
def get_seller(r):
    
    # get soup
    soup = BeautifulSoup(r)
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceConcession"]')
    if len(step1)>0:
        try:
            step2 = step1[0].select("strong")[0].text.split(":")
            return step2[-1].strip()
        except:
            pass
    return np.nan

In [778]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller = get_seller(readable_html)
g_seller

'SELECT MACHINES'

### get seller name

In [779]:
def get_seller_name(r):
    
    # get soup
    soup = BeautifulSoup(r)
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceConcession"]')
    if len(step1)>0:
        try:
            step2 = step1[0].select('div[id*="IDadresse"]')
            step3 = step2[0].text.replace("\t", " ").split("\n")
            for k in step3:
                if k.find("Votre contact") >= 0:
                    return k.split(":")[-1].strip()
        except:
            pass
    return np.nan

In [780]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller_name = get_seller_name(readable_html)
g_seller_name

'Jocelyn'

### get vehicle release date

In [781]:
def get_release(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Date de mise") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [782]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_rel = get_release(readable_html)
g_rel

'29/05/2018'

### get vehicle mileage

In [783]:
def get_mileage(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Kilom") >= 0:
                    try:
                        return float(re.findall("\d+", k.select('td')[1].text)[0])
                    except:
                        return np.nan
    else:
        return np.nan

In [784]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_mileage = get_mileage(readable_html)
g_mileage

12057.0

### get engine capacity

In [785]:
def get_capa(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Cylind") >= 0:
                    try:
                        return float(re.findall("\d+", k.select('td')[1].text)[0])
                    except:
                        return np.nan
    else:
        return np.nan

In [786]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_capa = get_capa(readable_html)
g_capa

125.0

### get engine type

In [787]:
def get_engine_type(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Motori") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [788]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_et = get_engine_type(readable_html)
g_et

'Essence'

### get fiscal power

In [793]:
def get_fiscal_power(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Puissance") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [794]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_fp = get_fiscal_power(readable_html)
g_fp

### get guarantee

In [795]:
def get_guarantee(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Garant") >= 0:
                    try:
                        return k.select('td')[1].text.replace("\n", "").replace("\t", "").strip()
                    except:
                        return np.nan
    else:
        return np.nan

In [796]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_guar = get_guarantee(readable_html)
g_guar

"3 mois jusqu'au 01/01/1970"

### get hand

In [797]:
def get_hand(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Première") >= 0:
                    try:
                        return "First hand"
                    except:
                        return np.nan
    else:
        return np.nan

In [798]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_hand = get_hand(readable_html)
g_hand

### get chassis

In [799]:
def get_chassis(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Chassis") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [800]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_chassis = get_chassis(readable_html)
g_chassis

### get comment

In [801]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

In [802]:
def get_comment(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceDescription"]')
    if len(step1)>0:
        try:
            return cleanhtml(step1[0].text.replace("\n", " ").replace("\t", " ").replace("  ", " ").strip())
        except:
            return ""
    return ""

In [480]:
with codecs.open(file_check_single, 'r', encoding='utf-8', errors='ignore') as f:
    readable_html = f.read()
    g_comment = get_comment(readable_html)
g_comment

'Description Equipements de série : ABS, Poignées chauffantes, Prise 12V, Top case 2 casques, Valises, Version GTEquipements optionnels inclus : Silencieux AkrapovicRemarques entretien : Aucun consommable à prévoir, Factures d‘entretien, Révision faiteServices concession : Démarches administratives sur place, Essai sur rendez-vous, Financement & Assurance possibles'

### Save image functions

In [481]:
def get_images(r, uniq_id):
    
    # get images url list
    image_list = []
    
    # get soup
    soup_ = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup_.select('div[class*="CLSlide-group"]')
    
    if len(step1)>0:
        step2 = step1[0].select("a")
        if len(step2) > 0:
            for k in step2:
                if "href" in k.attrs:
                    image_list.append(k.attrs['href'])

    # save images
    k=1
    for image_url in image_list[0:3]:
        image_name = f'images/{uniq_id}-{k}.jpg'
        if os.path.isfile(image_name) is False:
            img_data = requests.get(image_url).content
            with open(image_name, 'wb') as handler:
                handler.write(img_data)
            try:
                image = Image.open(image_name) 
                ratio = image.size[0] / image.size[1]
                image = image.resize((300,int(300/ratio)))
                image.save(f'images/{uniq_id}-{k}.jpg',optimize = True, quality = 70)
            except:
                pass
        k+=1
        
    return

In [431]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
    get_images(readable_html, get_uniq_id(g_url))

https://www.fulloccaz.com/nas/commun/annonces//a6b9fff993692e166f34a41bd2eea0ef8c8b833d/photo-annonce-kymco-ak-550_5f4e6f385481d443057694.jpg
https://www.fulloccaz.com/nas/commun/annonces//a6b9fff993692e166f34a41bd2eea0ef8c8b833d/photo-annonce-kymco-ak-550_5f4e6f3d8028a305583159.jpg
https://www.fulloccaz.com/nas/commun/annonces//a6b9fff993692e166f34a41bd2eea0ef8c8b833d/photo-annonce-kymco-ak-550_5f4e6f42683ca632103229.jpg


<div style="color:orangered; font-weight:700; font-size:20pt">TEST 100 ANNONCES</div>

In [825]:
counter = 1
list_test = []
for single_ in [file for file in os.listdir("annonces") if file.endswith("html")]:
    with open(f"annonces/{single_}", 'r') as f:
        readable_html = f.read()
        g_X = get_postal(readable_html)
        print(single_, counter, g_X)
        counter+=1

fulloccaz-2020182-2021-06-07_22h36.html 1 56600
fulloccaz-2050240-2021-06-07_22h36.html 2 56600
fulloccaz-2050254-2021-06-07_22h36.html 3 56600
fulloccaz-2023856-2021-06-07_22h37.html 4 38400
fulloccaz-2048318-2021-06-07_22h37.html 5 51000
fulloccaz-2050051-2021-06-07_22h36.html 6 56600
fulloccaz-2047183-2021-06-07_22h38.html 7 64600
fulloccaz-1992853-2021-06-07_22h37.html 8 56600
fulloccaz-2049632-2021-06-07_22h38.html 9 56600
fulloccaz-2048838-2021-06-07_22h37.html 10 51000
fulloccaz-2026239-2021-06-07_22h37.html 11 92600
fulloccaz-2050222-2021-06-07_22h36.html 12 56600
fulloccaz-2050228-2021-06-07_22h37.html 13 56600
fulloccaz-2041214-2021-06-07_22h38.html 14 92600
fulloccaz-2031399-2021-06-07_22h38.html 15 56600
fulloccaz-2049139-2021-06-07_22h37.html 16 56600
fulloccaz-2049703-2021-06-07_22h38.html 17 86000
fulloccaz-2049146-2021-06-07_22h38.html 18 56600
fulloccaz-2041221-2021-06-07_22h37.html 19 56600
fulloccaz-2050145-2021-06-07_22h38.html 20 56600
fulloccaz-2044036-2021-06-07_

## Extract 1 announce

### function to check already existing announce in the df

In [506]:
def add_the_announce(df, uniq_id, price):
    if int(uniq_id) in list(df["unique id"]):
        index_ = df.index[df['unique id'] == int(uniq_id)].tolist()[0]

        # if an announce with same uniq id but different price is found, return true and add
        # or return False and skip
        return df.iloc[index_]["price"]!=price

    # else, return true and add
    return True

In [627]:
def process_announce(file_name):
    """ this function add an announce content from fulloccaz.com to the dataframe """
    
    # load csv if exists or starting from template
    data_fulloccaz = "../../../tresboncoin/data/fulloccaz.csv"
    #
    if os.path.isfile(data_fulloccaz) is False:
        pass
    else:
        data = pd.read_csv(data_fulloccaz)
    
    df = announce_template.copy()
    
    # fill df with defined functions
    with open(f"annonces/{file_name}", 'r') as f:
        readable_html = f.read()
        
        url_ = get_url(readable_html)
        uniq_id = get_uniq_id(url_)
        price_ = get_price(readable_html)
        
        print(file_name)
        
        if isinstance(url_, str):
            if add_the_announce(data, uniq_id, price_):
                df["url"] = url_
                df["unique id"] = uniq_id
                df["date_scrapped"] = datetime.datetime.now().strftime("%Y/%m/%d - %Hh%M")
                df["announce_publication_date"] = get_publication_date(readable_html)
                df["vehicle brand"] = get_brand_and_model(readable_html)
                df["vehicle type"] = get_type(readable_html)
                df["color"] = get_color(readable_html)
                df["price"] = price_
                df["city"] = get_city(readable_html)
                df["postal code"] = get_postal(readable_html)
                df["seller"] = get_seller(readable_html)
                df["seller_name"] = get_seller_name(readable_html)
                df["vehicle release date"] = get_release(readable_html)
                df["mileage"] = get_mileage(readable_html)
                df["motorisation"] = get_engine_type(readable_html)
                df["fiscal power"] = get_fiscal_power(readable_html)
                df["guarantee"] = get_guarantee(readable_html)
                df["hand"] = get_hand(readable_html)
                df["chassis"] = get_chassis(readable_html)
                df["engine capacity [CC]"] = get_capa(readable_html)
                df["comments"] = get_comment(readable_html)
        
                # save images
                get_images(readable_html, get_uniq_id(get_url(readable_html)))
            
                # concatenate to csv and write
                try:
                    data = pd.concat([data, df], axis=0)
                    data.to_csv(path_or_buf = data_fulloccaz, index=False)
                except:
                    df.to_csv(path_or_buf = data_fulloccaz, index=False)
            else:
                print("id: " + str(uniq_id) + " skipped")
            # deplacer le fichier html traité dans le vault
            subprocess.run(["mv", f"annonces/{file_name}", "vault"])
        else:
            subprocess.run(["mv", f"annonces/{file_name}", f"annonces/{file_name}_bad"])
    return

In [628]:
list(os.listdir("annonces")[0:2])

['fulloccaz-2016872-2021-06-06_12h22.html',
 'fulloccaz-2017381-2021-06-06_20h29.html']

In [ ]:
for single_ in [file for file in os.listdir("annonces") if file.endswith(".html")]:
    process_announce(single_)

In [6]:
data_moto = "../../../tresboncoin/data/fulloccaz.csv"
data = pd.read_csv(data_moto)
data.tail(10)

,url,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,color,price,city,postal code,seller,seller_name,vehicle release date,mileage,motorisation,fiscal power,guarantee,hand,chassis,engine capacity [CC],comments
4679,http://www.fulloccaz.com/fiches/index/2050599-...,2050599.0,2021/06/08 - 10h57,08/06/2021,TRIUMPH BONNEVILLE T100 BLACK,Routière,Noir,7190.0,MONTPELLIER,34000.0,PASCAL MOTO TRIUMPH,Yannick,05/12/2015,21389.0,4T,68 CV,12 mois,NaN,NaN,865.0,Description SUPERBE TRIUMPH BONNEVILLE T100 ...
4680,http://www.fulloccaz.com/fiches/index/2048844-...,2048844.0,2021/06/08 - 10h57,07/06/2021,PIAGGIO GTS 300 ACCIDENTE REF 48048,Scooter,Gris,1495.0,127 VITROLLES,11313.0,POLE POSITION MOTOS,Jean Christophe,18/06/2016,20000.0,4T,NaN,NaN,NaN,NaN,300.0,Description Maxiscooter accidenté PIAGGIO GT...
4681,http://www.fulloccaz.com/fiches/index/2050591-...,2050591.0,2021/06/08 - 10h57,08/06/2021,YAMAHA WR250F,Enduro,Bleu,3490.0,ANNONAY,7100.0,NICOLAS MOTOS,Nicolas,10/03/2003,5960.0,4T,NaN,NaN,NaN,NaN,250.0,Description Moto réviséeReprise possible KTM...
4682,http://www.fulloccaz.com/fiches/index/2044048-...,2044048.0,2021/06/08 - 10h57,29/04/2021,MOTO-GUZZI BELLAGIO,Custom,Noir,6990.0,TREGUEUX,22950.0,DESTINATION MOTO,Service Commercial,05/07/2011,34187.0,4T,NaN,6 mois jusqu'au 01/01/1970,NaN,NaN,940.0,"Description Options incluses :Bulle haute, F..."
4683,http://www.fulloccaz.com/fiches/index/2042642-...,2042642.0,2021/06/08 - 10h57,08/06/2021,AFRICA TWIN ADVENTURE SPORT,Trail,Argent,12790.0,PERPIGNAN,66000.0,POLE MOTO PASSION PERPIGNAN,Service commercial,28/05/2019,10800.0,4T,95 CV,12 mois,NaN,NaN,1000.0,Description HONCA CRF1000AFRICATWIN 1000 ADV...
4684,http://www.fulloccaz.com/fiches/index/2021287-...,2021287.0,2021/06/08 - 10h57,08/06/2021,PEUGEOT METROPOLIS 400 BLACK EDITION,Scooter,Noir,7500.0,TETEGHEM,59229.0,WANTED BIKE,Fred,09/04/2019,850.0,4T,5 CV,Constructeur jusqu'au 08/04/2022,First hand,3 roues,400.0,Description PEUGEOT METROPOLIS 400 BLACK EDI...
4685,http://www.fulloccaz.com/fiches/index/2005518-...,2005518.0,2021/06/08 - 10h57,07/06/2021,KYMCO DOWNTOWN 350I EXCLUSIVE MOTORS 54 1ERE M...,Scooter,Gris,4500.0,DOMMARTIN LES TOUL,54200.0,MOTORS 54,NICOLAS,28/11/2019,1190.0,4T,4 CV,Constructeur jusqu'au 28/11/2022,First hand,NaN,350.0,Description Equipements de série : Peinture métal
4686,http://www.fulloccaz.com/fiches/index/2050600-...,2050600.0,2021/06/08 - 10h57,08/06/2021,YAMAHA TRACER 900 GT,Routière,Argent,11200.0,BOURG EN BRESSE,1000.0,MOTOAIN,Stephane,07/01/2021,500.0,4T,115 CV,NaN,NaN,NaN,850.0,Description Concessionnaire Yamaha vend trac...
4687,http://www.fulloccaz.com/fiches/index/2050198-...,2050198.0,2021/06/08 - 10h57,07/06/2021,Belle moto de collection SUZUKI RG GAMMA 500 d...,Sportive,Blanc,22590.0,127 VITROLLES,11313.0,POLE POSITION MOTOS,Jean Christophe,01/01/1986,12514.0,4T,NaN,NaN,NaN,NaN,500.0,"Description Collector absolu, moteur mythiqu..."
4688,http://www.fulloccaz.com/fiches/index/2048639-...,2048639.0,2021/06/08 - 10h57,08/06/2021,DUCATI SCRAMBLER 800,Routière,Jaune,6990.0,MONTPELLIER,34000.0,PASCAL MOTO TRIUMPH,Yannick,26/01/2016,13312.0,4T,74 CV,12 mois,NaN,NaN,800.0,Description DUCATI SCRAMBLER 800. SUPERBE . ...


In [3]:
data.shape

(4689, 21)

In [4]:
data.iloc[4624]["url"]

'https://www.fulloccaz.com/fiches/index/2043830-occasion-sherco-sef-r-500-factory-2020-motors-54?&Marque=SHERCO'